# Implementing a `Gradient descent` algorithm from Scratch

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

**`Gradient descent` is a popular optimization algorithm used in machine learning to minimize the cost function of a model. The cost function is a mathematical function that measures how well the model fits the training data.**

**The goal of the algorithm (_the optimizer_) is to find the set of parameters that minimize the cost function. `Gradient descent` works by iteratively adjusting the parameters of the model in the opposite direction of the gradient of the cost function with respect to the parameters.**

**There are several variations of `gradient descent`, including `batch gradient descent`, `stochastic gradient descent`, and `mini-batch gradient descent`, each with its own advantages and disadvantages depending on the problem at hand.**

![gradient-descent](https://upload.wikimedia.org/wikipedia/commons/a/a3/Gradient_descent.gif)

**In this notebook, we will implement a `gradient descent` algorithm from scratch. Let us first create an artificial dataset. For this tutorial, our data distribution is going to be a ~ `mystery function` ~.**


In [1]:
import numpy as np
import plotly.graph_objects as go

x = np.random.randn(100)*2

noise = np.random.normal(-1, 1, 100) *0.15

y = np.sin(x) + noise

fig = go.Figure(data=go.Scatter(
    x=x, y=y, mode='markers', name='Mystery Function'))

fig.update_layout(template='plotly_dark',
                  title='Mystery Function',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

**If you read the cell above, or you just looked at the plotted graph, you already know the function we want to approximate. However, let us pretend we don't.**

**Here are some possible models that could (if we didn't know what it was) explain the function above:**

$$y_1 = f(x) = cos(x - 3)$$
$$y_2 = f(x) = cos(2x + 2)$$
$$y_3 = f(x) = cos(x + 2)$$
$$y_4 = f(x) = cos(2x - 2)$$

**Let us see how these models compare to our mystery function.**

In [2]:
def model(w, x):
    """
    Compute the output of the cosine function given 
    the input x and the weight vector w.

    Args:
        w (numpy.ndarray): A 1D numpy array of size 2 
    containing the weight parameters of the model.
        x (numpy.ndarray): A 1D numpy array representing 
    the input to the model.

    Returns:
    numpy.ndarray: A 1D numpy array representing the 
    output of the model for the given input.

    """
    return np.cos(w[0] * x + w[1])


x_test = np.linspace(x.min()-.1, x.max()+.1, 500)

y_1 = model([1, -3], x_test)
y_2 = model([2, 2], x_test)
y_3 = model([1, 2], x_test)
y_4 = model([2, -2], x_test)

fig = go.Figure(data=go.Scatter(
    x=x, y=y, mode='markers', name='Mystery Function'))

fig.add_trace(go.Scatter(x=x_test, y=y_1, name='f(x) = cos(x - 3)'))
fig.add_trace(go.Scatter(x=x_test, y=y_2, name='f(x) = cos(2x + 2)'))
fig.add_trace(go.Scatter(x=x_test, y=y_3, name='f(x) = cos(x + 2)'))
fig.add_trace(go.Scatter(x=x_test, y=y_4, name='f(x) = cos(2x - 2)'))

fig.update_layout(template='plotly_dark',
                  title='Guess Functions',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

**Besides a visual inspection, a more rigorous way to measure how well our models fit this mystery fuction is by using a cost/loss function, like MSE (Mean Squared Error).**

**Mean Squared Error (MSE) is a widely used metric in machine learning to measure the performance of regression models. It is the average squared difference between the predicted values and the actual values. To calculate the MSE, we take the difference between each predicted value and the actual value, square it, and then take the average of all the squared differences. The result is a single number that represents how well the model fits the data.**

$$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{y}_i)^2$$

**where:** 
- **$n$ is the number of samples.**
- **$y_i$ is the actual value of the $i^{th}$ sample.**
- **$\hat{y}_i$ is the predicted value of the $i^{th}$ sample.**

**Or, simply put in `Numpy`: `np.mean((y - np.sin(w[0] * x + w[1]))**2)`**.


In [7]:
def mse(actual, predicted):
    """
    Calculate the mean squared error between the actual and predicted values.

    Args:
        actual (list or numpy array): List or numpy array of actual values.
        predicted (list or numpy array): List or numpy array of predicted values.

    Returns:
        float: The mean squared error between the actual and predicted values.
    """
    actual = np.array(actual)
    predicted = np.array(predicted)
    differences = np.subtract(actual, predicted)
    squared_differences = np.square(differences)
    return squared_differences.mean()

fig = go.Figure([go.Bar(x=['y_1', 'y_2', 'y_3', 'y_4'], 
                        y=[abs(mse(y, np.cos(x - 3))),
                        abs(mse(y, np.cos(2*x + 2))),
                        abs(mse(y, np.cos(x + 2))),
                        abs(mse(y, np.cos(2*x - 2)))],
                        )])

fig.update_traces(texttemplate='%{y:.2f}')

fig.update_layout(template='plotly_dark',
                  title_text='Model Loss Comparison',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')

fig.show()

**According to MSE, $y_1$ is the best model, and $y_3$ is the worst.**

**Before going to `gradient descent` let us try another solution: _brute force search_.**

**Brute force search is a simple and straightforward method of searching for a solution to a problem by checking every possible option in a systematic way until the correct solution is found. It involves evaluating every possible solution, no matter how inefficient or unlikely it may be, until the correct one is discovered.**

**Let's generate random values between -3 and 3 and use them to create a model, i.e., the values of `w0` and `w1`.**

In [9]:
import pandas as pd 

w0_val = np.linspace(-3, 3, 50)
w1_val = np.linspace(-3, 3, 50)

loss_log = []

for w0 in w0_val:
    for w1 in w1_val:
        loss_log.append([w0, w1, mse(np.sin(x), np.cos(w0 * x + w1))])

loss_log_df = pd.DataFrame(loss_log, columns=['w0', 'w1', 'MSE'])

w0_BF = loss_log_df.loc[loss_log_df['MSE'].idxmin(), ['w0', 'w1']]['w0']
w1_BF = loss_log_df.loc[loss_log_df['MSE'].idxmin(), ['w0', 'w1']]['w1']

brute_force_model = model([w0_BF, w1_BF], x_test)

fig = go.Figure(data=go.Scatter(
    x=x, y=y, mode='markers', name='Mystery Function'))

fig.add_trace(go.Scatter(x=x_test, y=brute_force_model,
              name=f'f(x) = sin({w0_BF:.2f} * x + {w1_BF:.2f})'))

fig.update_layout(template='plotly_dark',
                  title='Brute Force Model',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')

fig.show()

**While brute force search may be effective for small problem sets, it can become computationally infeasible for larger problem sets due to the sheer number of possible solutions that must be evaluated.**

**Now that we logged the MSE value of our many brute force models, let us take a look at the loss surface of our problem.**

In [10]:
fig = go.Figure()

fig.add_trace(go.Surface(x=w0_val, y=w1_val,
                         z=loss_log_df['MSE'].to_numpy().reshape((len(w0_val), len(w1_val)))))

fig.update_layout(margin=dict(l=0, r=0, t=70, b=0),
                  template='plotly_dark',
                  title='Loss Function Landscape - 3D',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

fig = go.Figure()

fig.add_trace(go.Contour(
    x=loss_log_df['w0'], y=loss_log_df['w1'], z=loss_log_df['MSE']))

fig.update_layout(margin=dict(l=0, r=0, t=70, b=0),
                  template='plotly_dark',
                  title='Loss Function Landscape - 2D',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

**Our loss surface has two global minima (two optimal solutions). The regions near the global minima are convex ("U" shaped) and we could use this "inclination" (_the sign and magnitude of the derivative_) to our favor. This is what `gradient descent` is all about.**

**The formula for gradient descent is:**

$$ \theta_{j+1} = \theta_j - \alpha \nabla J(\theta_j) $$

**where:** 

- **$\theta_j$ is the vector of model parameters at iteration $j$.** 
- **$\alpha$ is the learning rate.** 
- **$J(\theta)$ is the loss/cost function.** 
- **$\nabla J(\theta)$ is the gradient of the cost function concerning the model parameters.** 

**The gradient is a vector of partial derivatives of the cost function concerning each parameter. The gradient descent algorithm takes steps in the direction of the negative gradient until it reaches a minimum of the cost function.**

**For this problem, our loos function will be MSE, since we are trying to predict real values.**

**Calculating the derivative of our function in this toy example is simple since the derivative of our ~ mystery function ~ is well known ($Cos$).**

**To help you understand that the gradient basically points to the direction where the optimizer should "move the parameters of the model", let us plot arrows at every point of our loss landscape, representing the direction/magnitude of the gradient.**

In [11]:
import plotly.figure_factory as ff

def gradient(w):
    """
    Computes the gradient of the mean squared error loss function 
    with respect to the weights wfor a cosine model with input 
    feature x and target variable y.

    Parameters
    ----------
    w : array-like of shape (2,)
        The weights of the cosine model, where w[0] is the frequency 
        parameter and w[1] is the phase parameter.

    Returns
    -------
    gradient : ndarray of shape (2,)
        The gradient of the mean squared error loss function 
        with respect to the weights w.

    """
    g0 = -np.mean(2 * (y - np.cos(w[0] * x + w[1]))
                  * - np.sin(w[0]*x + w[1])*x)
    g1 = -np.mean(2 * (y - np.cos(w[0] * x + w[1])) * - np.sin(w[0]*x + w[1]))
    return np.array([g0, g1])


loss_grad_df = loss_log_df.join(loss_log_df[['w0', 'w1']].apply(lambda w: gradient(
    w), axis=1, result_type='expand').rename(columns={0: 'g0', 1: 'g1'}))

fig = go.Figure()

fig = ff.create_quiver(x=loss_grad_df['w0'], y=loss_grad_df['w1'],
                       u=loss_grad_df['g0'], v=loss_grad_df['g1'],
                       line_width=2, line_color='white',
                       scale=0.1, arrow_scale=.2)

fig.add_trace(go.Contour(x=loss_grad_df['w0'], y=loss_grad_df['w1'], z=loss_grad_df['MSE']))

fig.update_layout(margin=dict(l=0, r=0, t=70, b=0),
                  xaxis_range=[w0_val.min(), w0_val.max()],
                  yaxis_range=[w1_val.min(), w1_val.max()],
                  template='plotly_dark',
                  title='Loss Function Landscape with Gradient - 2D',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')

fig.show()


**Our loss landscape was two possible global minimums and many local minima and saddle points that can get our gradient stuck. Thus, we will use a common trick in ML. We will decrease the size of the gradient step by making or learning rate shrink as epochs pass.**

**Starting our optimizer from two different points, we will track its travel across the loss landscape. In the end, our model will find the most suited parameters to model our ~ mystery function ~.**

In [12]:
from IPython.display import Markdown
import math


def gradient_descent(w_0, lr=lambda t: 1./(t+1.), nepochs=10):
    """
    Performs gradient descent optimization to minimize a function 
    with the given initial weights.

    Args:
    - w_0 (numpy.ndarray): Initial weights.
    - lr (function): Learning rate function, takes the current 
        epoch as input and returns a scalar. Defaults to 1./(t+1.), 
        where t is the current epoch.
    - nepochs (int): Number of epochs to train.

    Returns:
    - numpy.ndarray: A numpy array of shape (nepochs+1, n_weights), 
        where n_weights is the number of weights in the model. 
        Each row contains the weights after an epoch of training.
    """
    w = w_0.copy()
    values = [w]
    for t in range(nepochs):
        w = w - lr(t) * gradient(w)
        values.append(w)
    return np.array(values)


GD_VALUES = gradient_descent(np.array([.8, 1.5]),
                             nepochs=400,
                             lr=lambda t: 1./np.sqrt(t+1.))

fig = go.Figure()

fig.add_trace(go.Contour(
    x=loss_grad_df['w0'], y=loss_grad_df['w1'], z=loss_grad_df['MSE']))

fig.add_trace(go.Scatter(x=GD_VALUES[:, 0], y=GD_VALUES[:, 1], name='Gradient Path', mode="markers+lines",
                         line=go.scatter.Line(color='white')))

fig.update_layout(margin=dict(l=0, r=0, t=70, b=0),
                  xaxis_range=[w0_val.min(), w0_val.max()],
                  yaxis_range=[w1_val.min(), w1_val.max()],
                  template='plotly_dark',
                  title='Loss Function Gradient Descent Path - 2D',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')

fig.show()

fig = go.Figure()

fig.add_trace(
    go.Surface(x=w0_val, y=w1_val,
               z=loss_log_df['MSE'].to_numpy().reshape((len(w0_val), len(w1_val)))))

fig.add_trace(
    go.Scatter3d(x=GD_VALUES[:, 1], y=GD_VALUES[:, 0], z=[np.mean((y - model(w, x))**2) for w in GD_VALUES],
                 line=dict(color='white')))

fig.update_layout(margin=dict(l=0, r=0, t=70, b=0),
                  template='plotly_dark',
                  title='Loss Function Gradient Descent Path - 3D',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')

fig.show()


GD_VALUES = gradient_descent(np.array([-.8, -1.5]),
                             nepochs=400,
                             lr=lambda t: 1./np.sqrt(t+1.))

fig = go.Figure()

fig.add_trace(go.Contour(
    x=loss_grad_df['w0'], y=loss_grad_df['w1'], z=loss_grad_df['MSE']))

fig.add_trace(go.Scatter(x=GD_VALUES[:, 0], y=GD_VALUES[:, 1], name='Gradient Path', mode="markers+lines",
                         line=go.scatter.Line(color='white')))

fig.update_layout(margin=dict(l=0, r=0, t=70, b=0),
                  xaxis_range=[w0_val.min(), w0_val.max()],
                  yaxis_range=[w1_val.min(), w1_val.max()],
                  template='plotly_dark',
                  title='Loss Function Gradient Descent Path - 2D',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')

fig.show()


fig = go.Figure()
fig.add_trace(
    go.Surface(x=w0_val, y=w1_val,
               z=loss_log_df['MSE'].to_numpy().reshape((len(w0_val), len(w1_val)))))
fig.add_trace(
    go.Scatter3d(x=GD_VALUES[:, 1], y=GD_VALUES[:, 0], z=[np.mean((y - model(w, x))**2) for w in GD_VALUES],
                 line=dict(color='white')))

fig.update_layout(margin=dict(l=0, r=0, t=70, b=0),
                  template='plotly_dark',
                  title='Loss Function Gradient Descent Path - 3D',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')

fig.show()


display(Markdown(f'''
## Final values for $w$ and $b$ after gradient descent:

- $w = {GD_VALUES[-1][0]:.5f}$.
- $b = {GD_VALUES[-1][1]:.5f}$.

### Final Model:

$$\cos({GD_VALUES[-1][0]:.5f} \\times x + {GD_VALUES[-1][1]:.5f})$$

### Fun Fact:

$$\sin(x) = \cos(x - \pi /2)$$
$$- \sin(x) = \cos(x + \pi /2)$$ 

**And that is why our loss-landscape has _two_ global minima.**

**Our model was trying to aproximate $\pi /2$, which is: ${math.pi/2:.5f}$**. 

**Our model got really close, the difference between ${GD_VALUES[-1][1]:.5f}$ and $\pi /2$ being ${- GD_VALUES[-1][1] - math.pi/2:.5f}$!**

'''
                 ))



## Final values for $w$ and $b$ after gradient descent:

- $w = 1.00549$.
- $b = -1.61042$.

### Final Model:

$$\cos(1.00549 \times x + -1.61042)$$

### Fun Fact:

$$\sin(x) = \cos(x - \pi /2)$$
$$- \sin(x) = \cos(x + \pi /2)$$ 

**And that is why our loss-landscape has _two_ global minima.**

**Our model was trying to aproximate $\pi /2$, which is: $1.57080$**. 

**Our model got really close, the difference between $-1.61042$ and $\pi /2$ being $0.03962$!**



---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).
